In [1]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
import numpy as np
import os
import time
from tqdm import tqdm
%matplotlib inline

In [2]:
df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/sales.csv',index_col=0)
df = df.drop_duplicates()
print('There are %.0f unique records in the database now' % df.shape[0])
print('Total revenue: %.02f' % df.amount_paid_usd.sum())

There are 69754 unique records in the database now
Total revenue: 524908.48


In [ ]:
url

In [3]:
spotify_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/spotify_df.csv',index_col=0)

In [4]:
spotify_df.shape[0]

28050

In [5]:
matches = spotify_df[spotify_df.match == True]
matches.shape[0]

13737

In [39]:
artist_names = matches.bc_artist_name.values

df_matches = df[df.artist_name.isin(artist_names)]
urls_to_scrape = df_matches.url.values

for i,url in enumerate(urls_to_scrape):
    if url[0] == '/':
        urls_to_scrape[i] = 'https:' + url

Get urls already scraped:

In [38]:
urls_scraped = []
folder =  '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed/'
files = os.listdir(folder)

for i,file in enumerate(files):
    try:
        if not file[0] == '.':
            df_temp = pd.read_csv(os.path.join(folder,file))
            url = df_temp.album_url.values[0]
            if url[0] == '/':
                url = 'https:' + url
                print(url)
            
            urls_scraped.append(url)
    except:
        pass

print('%.0f albums have been scraped' % len(files))

2750 albums have been scraped


In [40]:
already = 0
notyet = 0
for url in urls_to_scrape:
    
    if url in urls_scraped:
        already+=1
    else:
        notyet+=1

In [41]:
already

8381

In [42]:
notyet

26932

# LOOP THROUGH BANDCAMP SPIDER

In [43]:
cd '/Users/shayneufeld/Dropbox/insight/pricecamp/bandcamp_spider'

/Users/shayneufeld/Dropbox/insight/pricecamp/bandcamp_spider


In [44]:
urls_to_scrape = np.asarray(urls_to_scrape)
urls_to_scrape = np.unique(urls_to_scrape)
urls_to_scrape = list(urls_to_scrape)

In [46]:
len(urls_to_scrape)

21078

In [50]:
url

'https://kalbells.bandcamp.com/album/ten-flowers'

In [65]:
for url in urls_to_scrape[10150:]:
    
    if (url not in urls_scraped):
        
        '''
        get input parameters
        '''
        if url[0] == 'h':
            album_url = url
        elif url[0] == '/':
            album_url = 'https:' + url
        else:
            print('URL BROKEN')
            #break
        artist_name = df[df.url==url].head(1).artist_name.values
        if not artist_name:
            artist_name = df[df.url==url[6:]].head(1).artist_name.values

        if type(artist_name) is not str:
            artist_name = artist_name[0]
            
        al_start = url.find('album')
            
        if al_start == -1:
            al_start = url.find('track')
        if al_start == -1:
            print(url)
        album_name = url[al_start+6:]
        
        #test to see if it has been scraped but has no entries:
        file_name = album_name + '-' + artist_name + '.csv'
        file_name = file_name.replace(' ','')
        
        if file_name in os.listdir(folder):
            print('%s already in folder' % file_name)
        elif 'merch' in album_url:
            print('%s is merch. Do not scrape' % file_name)
        else:
            print('%s new file!' % file_name)
            '''
            run bandcamp spider
            '''
            os.system("scrapy crawl bandcamp -a start_domain='%s' -a album_name='%s' -a artist_name='%s'" % (album_url,album_name,artist_name))
            #scrapy crawl bandcamp -a start_domain='https://andreabaroni.bandcamp.com/album/broken-space-ost' -a album_name='Broken Spaces' -a artist_name='Andrea Baroni'
            urls_scraped = np.append(urls_scraped,url)
            #np.savetxt('/Users/shayneufeld/Dropbox/insight/pricecamp/urls_scraped.csv',urls_scraped)
            print('%0f albums scraped' % len(os.listdir(folder)))
            time.sleep(1)

iota-Kardashev.csv new file!
2842.000000 albums scraped
songs-only-a-mother-can-love-Kardiac.csv new file!
2842.000000 albums scraped
praise-for-the-morning-KarenAlley.csv new file!
2842.000000 albums scraped
https://karenalley.bandcamp.com/merch/beyond-melodies-using-chords-to-add-harmony
://karenalley.bandcamp.com/merch/beyond-melodies-using-chords-to-add-harmony-KarenAlley.csv is merch. Do not scrape
in-my-room-KarenAtkins.csv new file!
2842.000000 albums scraped
in-my-own-time-KarenDalton.csv new file!
2843.000000 albums scraped
something-on-your-mind-KarenDalton.csv new file!
2843.000000 albums scraped
element-79-KarenJewels.csv new file!
2843.000000 albums scraped
speak-now-ep-Karikatura.csv new file!
2843.000000 albums scraped
can-you-feel-it-KariKimmel.csv new file!
2843.000000 albums scraped
all-i-want-ep-KarimaFrancis.csv new file!
2843.000000 albums scraped
la-mer-boire-KarimOuellet.csv new file!
2843.000000 albums scraped
oh-non-KarimOuellet.csv new file!
2843.000000 albums

2880.000000 albums scraped
22b-Kepler.csv new file!
2880.000000 albums scraped
tomorrow-2-Kerogen.csv new file!
2880.000000 albums scraped
jelly-Charmpit.csv new file!
2881.000000 albums scraped
rust-bone-KerrGriffin.csv new file!
2881.000000 albums scraped
two-brothers-and-a-ghost-Flagland.csv already in folder
total-victory-english-martyrs-4-TotalVictory.csv new file!
2881.000000 albums scraped
one-for-words-KestonCobblersClub.csv new file!
2882.000000 albums scraped
unreleased-2002-2012-Kettel.csv new file!
2883.000000 albums scraped
wingtip-Kettel.csv new file!
2884.000000 albums scraped
about-dragons-Kettel.csv new file!
2884.000000 albums scraped
when-can-Kettel&Secede.csv new file!
2885.000000 albums scraped
protagonista-KevinBorensztein.csv new file!
2885.000000 albums scraped
2014-everythings-going-along-as-usual-and-then-all-shit-breaks-loose-KevinDrumm.csv new file!
2886.000000 albums scraped
artificial-organ-KevinDrumm.csv new file!
2887.000000 albums scraped
june-KevinDrum

2937.000000 albums scraped
open-water-KingGizzard&TheLizardWizard.csv new file!
2937.000000 albums scraped
willoughbys-beach-2-KingGizzard&TheLizardWizard.csv already in folder
new-gordy-singles-KingGordy.csv new file!
2937.000000 albums scraped
king-leviathan-ep-KingLeviathan.csv new file!
2937.000000 albums scraped
troubles-by-the-score-KingofNone.csv new file!
2938.000000 albums scraped
borneo-mint-shave-KingPug.csv new file!
2938.000000 albums scraped
grocery-KingPug.csv new file!
2938.000000 albums scraped
water-pressure-KingPug.csv new file!
2938.000000 albums scraped
missin-missy-KingPuppy&TheCarnivore.csv new file!
2938.000000 albums scraped
sincere-KingReign.csv new file!
2938.000000 albums scraped
high-as-the-trees-KingRobby.csv new file!
2938.000000 albums scraped
henosis-KingsofImprog.csv new file!
2938.000000 albums scraped
true-soldiers-ep-Embassy.csv new file!
2938.000000 albums scraped
king-willow-KingWillow.csv new file!
2938.000000 albums scraped
created-in-the-image-

apparitions-Koroidia.csv new file!
2983.000000 albums scraped
none-so-brutal-edition-Korpse.csv new file!
2984.000000 albums scraped
volume-three-KosmischerLäufer.csv new file!
2985.000000 albums scraped
volume-two-KosmischerLäufer.csv new file!
2986.000000 albums scraped
https://kosmischerlaufer.bandcamp.com/merch/kosmischer-l-ufer-black-t-shirt
://kosmischerlaufer.bandcamp.com/merch/kosmischer-l-ufer-black-t-shirt-KosmischerLäufer.csv is merch. Do not scrape
mysticae-visiones-Kotebel.csv new file!
2986.000000 albums scraped
we-did-it-TheKount.csv new file!
2987.000000 albums scraped
you-glow-TheKount.csv new file!
2988.000000 albums scraped
utility-Kowton.csv new file!
2989.000000 albums scraped
pea-soup-Kowton.csv new file!
2990.000000 albums scraped
some-cats-Kowton.csv new file!
2990.000000 albums scraped
summer-ocean-Kr1z.csv new file!
2990.000000 albums scraped
lulacruza-comandante-kr3ture-remix-KR3TURE.csv new file!
2990.000000 albums scraped
endrhymes-Razen.csv new file!
2991.

3037.000000 albums scraped
remember-the-moon-LabCoast.csv new file!
3038.000000 albums scraped
perfection-is-terrible-GoAskAlice.csv new file!
3038.000000 albums scraped
the-night-and-i-are-still-so-young-TheHeavyBlinkers.csv new file!
3039.000000 albums scraped
le-mouvement-cr-e-la-mati-re-GrégorySalletQuintet.csv new file!
3039.000000 albums scraped
rois-de-nous-LaBronze.csv new file!
3040.000000 albums scraped
https://labyrinthear.bandcamp.com
://labyrinthear.bandcamp.com-LabyrinthEar.csv new file!
3040.000000 albums scraped
ingen-ny-dag-LaCasaFantom.csv new file!
3040.000000 albums scraped
las-cosas-Isasa.csv new file!
3041.000000 albums scraped
los-d-as-Isasa.csv new file!
3042.000000 albums scraped
ecos-de-la-selva-LaChamba.csv new file!
3042.000000 albums scraped
moliendo-caf-LaChamba.csv new file!
3042.000000 albums scraped
barb-s-MaraLakour.csv new file!
3043.000000 albums scraped
battle-meditation-KVD.csv new file!
3043.000000 albums scraped
equinoxe-Janeret.csv new file!
304

another-late-night-zero-7-Zero7.csv new file!
3093.000000 albums scraped
late-night-tales-belle-sebastian-vol-ii-Belle&Sebastian.csv new file!
3094.000000 albums scraped
late-night-tales-jamiroquai-Jamiroquai.csv new file!
3095.000000 albums scraped
late-night-tales-lafur-arnalds-ÓlafurArnalds.csv new file!
3096.000000 albums scraped
im-not-in-love-exclusive-10cc-cover-SongSung.csv new file!
3096.000000 albums scraped
the-imagination-of-thieves-LatinQuarter.csv new file!
3096.000000 albums scraped
fantasma-LaTortugaChina.csv new file!
3096.000000 albums scraped
the-great-mickey-mouse-swindle-LaTourette.csv new file!
3096.000000 albums scraped
cosmic-gravity-single-LauraBrehm.csv new file!
3097.000000 albums scraped
forever-for-sure-3-Laura&Greg.csv new file!
3097.000000 albums scraped
flowstate-LauraHickli.csv new file!
3097.000000 albums scraped
semper-femina-LauraMarling.csv new file!
3098.000000 albums scraped
i-adore-LauraMisch.csv new file!
3099.000000 albums scraped
plants-vs-zom

3152.000000 albums scraped
get-down-with-me-AngeloTinsley.csv new file!
3153.000000 albums scraped
long-way-back-lolita-YoungGunSilverFox.csv new file!
3154.000000 albums scraped
leggy-Leggy.csv new file!
3155.000000 albums scraped
leggy-audiotree-live-Leggy.csv new file!
3155.000000 albums scraped
im-gonna-destroy-that-boy-Leggy.csv new file!
3156.000000 albums scraped
--2-Легион.csv new file!
3156.000000 albums scraped
--5-Легион.csv new file!
3156.000000 albums scraped
dawn-of-genesis-Legionnaire.csv new file!
3157.000000 albums scraped
teac-life-Legowelt.csv new file!
3158.000000 albums scraped
unfolding-the-future-with-amateur-space-jazz-DannyWolfers.csv new file!
3159.000000 albums scraped
beyond-ur-self-Legowelt.csv new file!
3159.000000 albums scraped
phonkee-robots-vol-2-LeGreck.csv new file!
3159.000000 albums scraped
not-that-crazy-LehmannsBrothers.csv new file!
3159.000000 albums scraped
love-on-the-line-PeterLehndorff.csv new file!
3159.000000 albums scraped
lhlt009-rockbe

the-english-beach-BrokenEnglishClub.csv new file!
3207.000000 albums scraped
autosuggestion-SilentServant.csv new file!
3207.000000 albums scraped
breaking-the-flesh-BrokenEnglishClub.csv new file!
3208.000000 albums scraped
from-the-hip-EricCopeland.csv new file!
3209.000000 albums scraped
most-people-DeviceControl.csv new file!
3210.000000 albums scraped
piano-Terekke.csv new file!
3210.000000 albums scraped
plague-song-BrokenEnglishClub.csv new file!
3210.000000 albums scraped
rye-Randomer.csv new file!
3210.000000 albums scraped
gel-nde-Ferdinger.csv new file!
3210.000000 albums scraped
fastfall-Lifeformed.csv new file!
3211.000000 albums scraped
immerse-Lifeformed.csv new file!
3212.000000 albums scraped
li-f-e-ep-Lifeless.csv new file!
3212.000000 albums scraped
lucifer-is-calling-me-and-he-awaits-me-in-hell-Lifeless.csv new file!
3212.000000 albums scraped
i-know-what-im-not-LifeLessons.csv new file!
3213.000000 albums scraped
symphony-of-dread-single-Lifeless.csv new file!
3213

KeyboardInterrupt: 

In [64]:
list(urls_scraped)[:10]

['https://dreamcatalogue.bandcamp.com/album/--18',
 'https://telepathtelepath.bandcamp.com/album/--19',
 'http://lobstertheremin.com/track/--4',
 'https://grauenmetal.bandcamp.com/album/-',
 'https://zuriaakecn.bandcamp.com/album/-',
 'https://stefanvaldobrev.bandcamp.com/album/-',
 'https://m-a-d-e-s.bandcamp.com',
 'https://koantunes.bandcamp.com/track/06-koan-ananch-roeth-version',
 'https://filthycasualspod.bandcamp.com/album/10-games-to-play-before-you-die',
 'http://music.georgeclanton.com/album/100-electronica']

In [63]:
np.savetxt('/Users/shayneufeld/Dropbox/insight/pricecamp/urls_scraped',list(urls_scraped))

TypeError: Mismatch between array dtype ('<U157') and format specifier ('%.18e')

In [29]:
file_names = os.listdir(folder)

In [26]:
file_name

'/Users/shayneufeld/dropbox/insight/pricecamp/data/albums_fixed/that-positive-feeling-music-license-Alumo.csv'

In [32]:
file_name = album_name + '-' + artist_name + '.csv'
file_name

'that-positive-feeling-music-license-Alumo.csv'

In [33]:
file_name in file_names

True

In [ ]:
artist_name = df[df.url==url[6:]].head(1).artist_name.values
artist_name

In [31]:
file_names[100]

'adrift-in-sleepwakefulness-[Bucketheadland].csv'

In [ ]:
type(artist_name) is not str

In [ ]:
urls_scraped = []
folder =  '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed/'
files = os.listdir(folder)

for i,file in enumerate(files):
    try:
        if not file[0] == '.':
            df = pd.read_csv(os.path.join(folder,file))
            url = df.album_url.values[0]
            urls_scraped.append(url)
    except:
        pass

In [ ]:
urls_scraped

In [ ]:
d = {'url':urls_scraped}
urls_scraped_df = pd.DataFrame(data = d)
urls_scraped_df.to_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/urls_scraped.csv')